# Testing
This notebook is for visualizing and evaluating the model results from the different experiments that were evaluated in the thesis.
1. The dataset is created. If not already happened, the images are split into train, validation and test set and divided into smaller image patches. The JSON files that transform the data into COCO format are generated.
2. By specifying the experiment, for which the results shall be inspected, the respective trained model weights are downloaded and the configuration for the experiment is set.
3. Different options for visualizing the inference results are presented.
4. The model can be evaluated on one of the subsets.

## 0: Check Dependencies

In [ ]:
import sys
import os
import mmcv

ROOT_DIR = os.path.abspath("../MMDetection")
sys.path.append(ROOT_DIR)

# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__) # should be 2.25.0

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version
print(get_compiling_cuda_version()) # should be matching Pytorch's cuda version

## 1: Create Dataset

If the following values (except for the paths) are left unchanged, image patches and JSON files are created so that all experiments barring the one that uses image patches with a minimum scale of 150 pixels (modification \[b\]) can easily be reproduced.

In [ ]:
from DataInitialization import DataInitialization

# Path to the folder containing the original satellite images
INPUT_IMG_DIR = os.path.abspath("./data/Training_Images_RGB")
# Path to the folder containing the `.shp` annotations
ANNO_DIR = os.path.abspath("./data/Training_tree_polygons")
# Folder where train, val and test sets are created
IMG_DIR = os.path.dirname(INPUT_IMG_DIR)

# If sets are not created randomly, these indices mark the images belonging to the validation set
# and test set
val_images = [3, 8, 15, 16, 20, 28, 33, 35, 40, 43, 52, 60, 65, 68, 78, 79, 83, 89, 95, 98, 100, 107]
# For the reproduction of the reduced test set, the images 45, 91 and 99 need to be removed.
test_images = [5, 7, 17, 18, 24, 30, 41, 45, 47, 49, 51, 59, 62, 70, 76, 77, 81, 91, 99, 104, 105, 108]

# Width and height of the image patches. If a list is passed, patches of multiple sizes are created for
# the training set. For the validation and test set, only image patches of the first size are created. 
min_width = [300, 600]
min_height = [300, 600]
# Which subsets to create. 
subsets = ['train', 'val', 'test']

# Set up initialization of the datasets. If ``force`` is False, for each subset image patches and JSON
# files are only created if they do not exist thus far. If set to true, additional image patches are
# created and JSON files are regenerated, including all image patches.
ri = DataInitialization(img_dir=INPUT_IMG_DIR,
                          img_output_dir=IMG_DIR,
                          subsets=subsets,
                          force=False)

# Method to create image patches
ri.split_crop(min_width=min_width, min_height=min_height, random_split=False, val_images=val_images, test_images=test_images)
# Method to create JSON files in COCO format
ri.load_rwanda_data(anno_dir=ANNO_DIR)
ri.load_rwanda_data(anno_dir=ANNO_DIR, load_prefix="300", subsets=['train'])

## 2: Load Experiment for Testing
To reproduce the results of the different models evaluated in the thesis, their configurations and weights can be loaded by specifying the experiment name. A list of all the experiment_names can be generated with ``print_experiments()``.

I noticed that sometimes the weight downloads within JupyterLab from Github are very slow. Alternatively to the automatic download in JupyterLab, they can be downloaded manually from https://github.com/sbackmann/rwanda-instance/releases/tag/v2.25.0 and then need to be placed in their respective experiment directories (e.g. ./experiments/00_MaskR50_-/)

In [ ]:
from ModelInitialization import ModelInitialization

ModelInitialization.print_experiments()

In [ ]:
from ModelInitialization import ModelInitialization

# Specify which model shall be tested.
experiment = "MaskR50_acd"

mi = ModelInitialization(experiment=experiment)
model_dir = "./pretrained_models"

# Downloads the trained weights if not already downloaded
checkpoint = mi.load_experiment_weights()
# Loads configuration for chosen experiment
cfg = mi.load_config(anno_dir=ANNO_DIR, img_dir=IMG_DIR, checkpoint=checkpoint)

## 3: Visualize Inference Results

In [ ]:
from RwandaVisualization import RwandaVisualization

v = RwandaVisualization(cfg, checkpoint, subset='val')

### 3.1: Compare Inference Results to the Ground Truth
Plots the ground truth image patch next to the model's prediction. If no file name is specified, a random image from the subset is selected.

In [ ]:
# Either set img to None (random image) or specify image name (this must be in the subset)
img = None  # "300x300_image_65_1CNW1R.tif_tile_0-0.tif"
v.eval_image(img=img, save_fig=False)

### 3.2: Batch Ground Truth and Inference Generation
For a large scale comparison of ground truth and inference results, all ground truth annotations and predictions can be saved into a directory for further inspection.

In [ ]:
# Specify directory (does not have to be existent) to store ground truth images. Ground truths are only created if the specified directory is empty.
ground_truth_dir = "./visualization/ground_truths"
v.save_ground_truths(out_dir=ground_truth_dir)

# Specify directory (does not have to be existent) to store inference results. Results are only created if the specified directory is empty.
inference_dir = f"./visualization/{experiment}_inference"
v.save_inference_results(out_dir=inference_dir)

### 3.3: Show the Inference Results only
Alternatively, the full path to the image that should be inferenced can be specified. Then, only the prediction will be shown. This might be useful to test the model on new data that is neither in the train, val or test set.

In [ ]:
# Here, the full path to the image must be specified.
img = os.path.join(IMG_DIR, "val/300x300_image_65_1CNW1R.tif_tile_0-0.tif")
v.plot_inference_results(img)

## 4: Evaluate Model Metrics

In [ ]:
from ModelEvaluation import evaluate_rwanda
# Specify out_file if the results are to be saved.
out_dir = './visualization'
evaluate_rwanda(cfg, checkpoint, subset="test", out_dir=out_dir)

The model metrics as well as their training losses can also be inspected by looking at their tensorboard log files and thus without having to reproduce the reuslts. For this, tensorboard has to be opened with ``./experiments`` as the log directory.

In [ ]:
%load_ext tensorboard

In [ ]:
tensorboard --logdir=./experiments